<div style="text-align: left; font-weight: bold; font-size:30px;"> Study on the effect of periodic rebalancing of investment portfolios creator in different economies compared with the respective economic benchmark </div><hr>

An in-depth investigation into the impact of periodic rebalancing on investment portfolios composed of diverse assets, including bonds, equities, and other securities, across various national economies. This study aims to understand how regular adjustments in asset allocations influence portfolio performance relative to each country's unique economic benchmark. By comparing rebalanced portfolios to their respective national economic indicators, the research evaluates whether rebalancing strategies can enhance returns, mitigate risk, and adapt to economic fluctuations specific to each country’s market environment.

Before we start working we need to get the data and get the packages required for the processing. In this study, data from [Trading View](https://www.tradingview.com/) is used because [Trading View](https://www.tradingview.com/) as a platform gets and organizes data from different countries. In this way, we are able to get data that is otherwise not readily available and is also very costly to directly obtain.

#### Packages Required

In [ ]:
# Required for obtaining tradingview data
%pip install tradingview-datafeed
# Required for data processing
%pip install pandas
%pip install pandas-ta

#### Methodology
The project will be divided into two main parts, one will be used to manage the data and the other one will manage portfolio optimization models. These models will be initially applied to develop portfolios in different economies.

**Mean-Variance Optimization (MVO):**
Developed by Harry Markowitz, this model maximizes return for a given level of risk or minimizes risk for a given level of return by optimizing the mean and variance of portfolio returns.

**Capital Asset Pricing Model (CAPM):**
CAPM considers the relationship between expected return and risk by comparing portfolio return with a risk-free rate and the market return. It’s used to estimate the expected return of assets and helps in choosing assets that align with desired risk levels.

**Black-Litterman Model:**
This model incorporates investor views with the market equilibrium to produce a more customized and stable asset allocation. It modifies MVO by including subjective input on returns, making it valuable when integrating personal market perspectives.

**Risk Parity:**
Risk Parity allocates capital such that each asset contributes equally to the overall portfolio risk. This method often leads to diversified portfolios, particularly in terms of risk contributions rather than dollar amounts.

**Factor-Based Optimization:**
Using factors like size, value, momentum, and quality, this approach selects assets that score well on factors correlated with high returns. Factor-based models help to exploit anomalies and achieve superior risk-adjusted returns.

**Minimum Variance Portfolio:**
This is a low-risk strategy focused on minimizing portfolio volatility, typically by investing in low-correlation assets. This portfolio is optimal for risk-averse investors, balancing diversification and stability.

**Conditional Value at Risk (CVaR) Optimization:**
CVaR focuses on limiting extreme downside risk by optimizing for tail risk beyond a set threshold. Often used in stress testing, it is ideal for managing exposure to rare but severe market events.

**Sharpe Ratio Maximization:**
This model maximizes the Sharpe ratio (return-to-risk ratio) by optimizing the portfolio for the highest risk-adjusted return, enhancing returns relative to volatility. It’s useful when seeking efficient, high-return portfolios.

**Kelly Criterion:**
The Kelly Criterion is a growth-focused model used to determine optimal bet sizes, balancing growth with the probability of bankruptcy. It is particularly relevant in scenarios where capital preservation and compounding are essential.

**Multi-Objective Optimization:**
This advanced approach uses techniques like genetic algorithms and simulated annealing to balance multiple objectives, such as maximizing return, minimizing risk, and considering factors like ESG scores. This model is flexible for complex, customized constraints and goals.

#### Getting Economic Data
We need to decide what economies we need to target by their size and pirority, but focus will be put on the economies that are developed like **USA, China, India, Germany, France** these countries are initially selected. The asset classes will be selected on the availability of the same asset classes across all of these countries but mainly it will be stocks, commodities, and bonds. 

#### Process
Then these models are going to be fed data for all of these assets classes on a single pd.Pandas.dataframe and optimal weights will be allocated, this process will be done periodically like quarterly and this has been customizable. Then compared with the bechmark of the country (SPY in USA) returns as compared to buy and hold strategy and then methods are going to be shown in a matrix depicting (returns-benchmark) for every country.